<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/LLM_GPT2_QA_LoRA_DoRA_AdaLoRA_Multi_LoRA_MoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12

#GPT2_FFT

In [ ]:
import torch
import os
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from datasets import Dataset
import torch.nn.functional as F
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare 5 QA samples for training and validation
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = tokenizer.eos_token_id  # Ignore question and padding tokens

    inputs["labels"] = labels
    return inputs

# Convert samples to dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Convert to PyTorch DataLoader
batch_size = 2

def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["labels"] for item in batch])
    return input_ids, attention_mask, labels

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Cross-entropy loss function ignoring padding tokens
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model based on validation loss
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./gpt2-qa-best-loss-cml"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training and evaluation functions
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        # Save best model based on validation loss
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10)

# Load the best fine-tuned model for inference
model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Training Loss: 4.4203, Validation Loss: 1.5726
Best model saved at epoch 1 with validation loss: 1.5726
Epoch 2/10, Training Loss: 0.9883, Validation Loss: 0.9799
Best model saved at epoch 2 with validation loss: 0.9799
Epoch 3/10, Training Loss: 0.7012, Validation Loss: 0.6840
Best model saved at epoch 3 with validation loss: 0.6840
Epoch 4/10, Training Loss: 0.4719, Validation Loss: 0.4885
Best model saved at epoch 4 with validation loss: 0.4885
Epoch 5/10, Training Loss: 0.2782, Validation Loss: 0.4139
Best model saved at epoch 5 with validation loss: 0.4139
Epoch 6/10, Training Loss: 0.2091, Validation Loss: 0.3706
Best model saved at epoch 6 with validation loss: 0.3706
Epoch 7/10, Training Loss: 0.1939, Validation Loss: 0.3273
Best model saved at epoch 7 with validation loss: 0.3273
Epoch 8/10, Training Loss: 0.1228, Validation Loss: 0.2700
Best model saved at epoch 8 with validation loss: 0.2700
Epoch 9/10, Training Loss: 0.1057, Validation Loss: 0.2148
Best model sa

#GPT2_LoRA_HF_Trainer

In [ ]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModelForCausalLM, PeftModel


# Set seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configure LoRA adapters
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"]
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Verify trainable LoRA parameters

# Move model to device
model.to(device)

# Prepare QA samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore question and padding tokens

    inputs["labels"] = labels  # Ensure labels are correctly set
    return inputs

# Convert samples to dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Modified collate function for Trainer API compatibility
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Training Arguments
training_args = TrainingArguments(
    output_dir="./gpt2-lora-qa",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to=[],
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    tokenizer=tokenizer,
    data_collator=collate_fn  # Ensure correct batch formatting
)

# Start training
trainer.train()

# Save LoRA fine-tuned model
trainer.save_model("./gpt2-lora-qa")


# Load the base GPT-2 model first
base_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Load the LoRA adapter and merge it with the base model
model = PeftModel.from_pretrained(base_model, "./gpt2-lora-qa").to(device)

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding


# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-1d037cfc3a43>:110: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.493559
2,No log,1.483749
3,No log,1.472909
4,1.591900,1.460777
5,1.591900,1.448575
6,1.591900,1.436898
7,1.578200,1.425751
8,1.578200,1.415121
9,1.578200,1.404229
10,1.661400,1.393525


Q: What is the capital of France?
A: The capital of France is Paris.

Question


#Galore GPT2

In [ ]:
!pip -q install git+https://github.com/jiaweizzhao/GaLore

In [ ]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModelForCausalLM, PeftModel


# Set seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configure LoRA adapters
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"]
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Verify trainable LoRA parameters

# Move model to device
model.to(device)

# Prepare QA samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore question and padding tokens

    inputs["labels"] = labels  # Ensure labels are correctly set
    return inputs

# Convert samples to dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Modified collate function for Trainer API compatibility
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Training Arguments
training_args = TrainingArguments(
    output_dir="./gpt2-lora-qa",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to=[],
    optim="galore_adamw",
    optim_target_modules=["attn", "mlp"],
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    tokenizer=tokenizer,
    data_collator=collate_fn  # Ensure correct batch formatting
)

# Start training
trainer.train()

# Save LoRA fine-tuned model
trainer.save_model("./gpt2-lora-qa")


# Load the base GPT-2 model first
base_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Load the LoRA adapter and merge it with the base model
model = PeftModel.from_pretrained(base_model, "./gpt2-lora-qa").to(device)

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding


# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-1-29c238d475a2>:113: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Activated GaLoRE fine-tuning, depending on your model size and hardware, the training might take a while before starting. Please be patient !
base_model.model.transformer.h.0.attn has been matched but ignored as GaLore only supports linear layers. Please double check your `optim_target_modules`!
base_model.model.transformer.h.0.attn.c_attn has been matched but ignored as GaLore only supports linear layers. Please double check your `optim_target_modules`!
base_model.model.transformer.h.0.attn.c_attn.base_layer has been matched but ignored as GaLore only supports linear layers.

Epoch,Training Loss,Validation Loss
1,No log,1.500915
2,No log,1.499189
3,No log,1.497448
4,1.608500,1.495646
5,1.608500,1.493995
6,1.608500,1.492535
7,1.634200,1.491249
8,1.634200,1.490075
9,1.634200,1.488963
10,1.760000,1.487888


Q: What is the capital of France?
A: The capital of France


#GPT2_LoRA_From_Scratch

In [ ]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

# Set seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)

# Load tokenizer and GPT-2 model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = GPT2LMHeadModel.from_pretrained(model_name)
base_model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)

# LoRA Configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"]
)

# Wrap model with LoRA
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# QA Samples for Training & Validation
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Data Preprocessing Function
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore question and padding tokens

    inputs["labels"] = labels
    return inputs

# Convert to Dataset and Preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# DataLoader Collation Function
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

# DataLoader Creation
batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Model Saving Function
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./gpt2-lora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=5):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels for loss computation
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute Loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start Training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20)

from peft import PeftModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Define the path where the best model is saved
best_model_path = "./gpt2-lora-qa-best"

# Load the base GPT-2 model first
base_model = GPT2LMHeadModel.from_pretrained("gpt2").to("cuda" if torch.cuda.is_available() else "cpu")

# Load the LoRA adapter and merge it with the base model
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(best_model_path)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers using the fine-tuned model
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Epoch 1/20, Train Loss: 1.5768, Val Loss: 1.5593
✅ Best model saved at epoch 1 with validation loss: 1.5593
Epoch 2/20, Train Loss: 1.4712, Val Loss: 1.5486
✅ Best model saved at epoch 2 with validation loss: 1.5486
Epoch 3/20, Train Loss: 1.4281, Val Loss: 1.5368
✅ Best model saved at epoch 3 with validation loss: 1.5368
Epoch 4/20, Train Loss: 1.7802, Val Loss: 1.5240
✅ Best model saved at epoch 4 with validation loss: 1.5240
Epoch 5/20, Train Loss: 1.4239, Val Loss: 1.5100
✅ Best model saved at epoch 5 with validation loss: 1.5100
Epoch 6/20, Train Loss: 1.7470, Val Loss: 1.4947
✅ Best model saved at epoch 6 with validation loss: 1.4947
Epoch 7/20, Train Loss: 1.4601, Val Loss: 1.4768
✅ Best model saved at epoch 7 with validation loss: 1.4768
Epoch 8/20, Train Loss: 1.4138, Val Loss: 1.4569
✅ Best model saved at epoch 8 with validation loss: 1.4569
Epoch 9/20, Train Loss: 1.5599, Val Loss: 1.4359
✅ Best model saved at epoch 9 with validation loss: 1.4359
Epoch 10/20, Train Loss: 1.5

Inference with Batch Greedy Search

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer
from peft import PeftModel

def generate_answer(questions):
    """
    Generate answers for a batch of questions using greedy search.

    Args:
        questions (list of str): A list of two questions.

    Returns:
        list of str: The list of generated answers.
    """
    model.eval()

    # Create input texts for each question
    input_texts = [f"Question: {q}\nAnswer:" for q in questions]

    # Tokenize the batch with appropriate padding and truncation
    inputs = tokenizer(
        input_texts,
        return_tensors="pt",
        truncation=True,
        padding=True,  # pad to the longest sequence in the batch
        max_length=256
    ).to(model.device)

    print('input_ids:', inputs['input_ids'].shape)

    # Generate outputs using greedy search (do_sample=False ensures greedy decoding)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False  # Explicitly disable sampling to use greedy search
        )

    # Decode each generated sequence and extract the answer portion
    answers = []
    for output in outputs:
        decoded_text = tokenizer.decode(output, skip_special_tokens=True)
        # Assuming the output contains the prompt "Answer:" and the generated answer follows it.
        answer = decoded_text.split("Answer:")[-1].strip()
        answers.append(answer)

    return answers


# Path to the best fine-tuned model
best_model_path = "./llama-lora-qa-best"

# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned tokenizer
# tokenizer = LlamaTokenizer.from_pretrained(best_model_path)
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding


# Example usage with two sample questions:
questions = [
    "Provide only short answer of the question: Who created the Mona Lisa?",
    "Provide only short answer of the question: What is the capital of France?"
]
answers = generate_answer(questions)
for q, a in zip(questions, answers):
    print(f"Q: {q}\nA: {a}\n")


input_ids: torch.Size([2, 22])


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Q: Provide only short answer of the question: Who created the Mona Lisa?
A: Leonardo da Vinci.

Q: Provide only short answer of the question: What is the capital of France?
A: Paris.



#GPT2 MoELoRA

In [ ]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModelForCausalLM, PeftModel

# Set seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
base_model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
base_model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define multiple LoRA adapters to simulate MoE behavior
lora_experts = []
num_experts = 4  # Number of experts
for i in range(num_experts):
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=16,  # Low-rank adaptation
        lora_alpha=32,  # Scaling factor
        lora_dropout=0.1,  # Dropout rate
        target_modules=["c_attn", "c_proj"]  # Apply LoRA to attention layers
    )
    expert_model = get_peft_model(base_model, lora_config)
    lora_experts.append(expert_model)

# Function to select top-k experts dynamically
def select_experts(lora_experts, k=2):
    return random.sample(lora_experts, k)

# Custom MoE forward function
def moe_forward(input_ids, attention_mask):
    selected_experts = select_experts(lora_experts, k=2)
    outputs = [expert(input_ids=input_ids, attention_mask=attention_mask).logits for expert in selected_experts]
    return torch.stack(outputs, dim=0).mean(dim=0)  # Average the expert outputs

# Prepare QA samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Convert samples to dataset and preprocess
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)
    labels = inputs["input_ids"].copy()
    inputs["labels"] = labels
    return inputs

dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Training Arguments
output_dir = "gpt2-moe-lora-qa"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_dir="./logs",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to=[],
)

# Custom Trainer to use MoE forward function
class MoETrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # Added **kwargs to handle unexpected arguments
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)
        labels = inputs["labels"].to(device)
        logits = moe_forward(input_ids, attention_mask)
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, logits) if return_outputs else loss

# Define Trainer
trainer = MoETrainer(
    model=base_model,  # Base model (expert models are used inside MoE forward function)
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    tokenizer=tokenizer
)

# Start training
trainer.train()

# Save MoE LoRA fine-tuned model
for i, expert in enumerate(lora_experts):
    expert.save_pretrained(f"{output_dir}/expert_{i}")

# Load base model and fine-tuned MoE LoRA adapters
base_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
lora_experts = [PeftModel.from_pretrained(base_model, f"{output_dir}/expert_{i}").to(device) for i in range(num_experts)]

# Inference function
def generate_answer(question):
    base_model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    with torch.no_grad():
        output = moe_forward(inputs["input_ids"], inputs["attention_mask"])
        answer = tokenizer.decode(torch.argmax(output, dim=-1)[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-be1c9071026f>:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MoETrainer.__init__`. Use `processing_class` instead.
  trainer = MoETrainer(


Epoch,Training Loss,Validation Loss
1,No log,9.539512
2,No log,9.207100
3,No log,8.858803
4,No log,8.532180
5,No log,8.264081
6,No log,8.061941
7,No log,7.901735
8,No log,7.784018
9,No log,7.711033
10,No log,7.681858


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Q: What is the capital of France?
A: ,
 is the difference of the?
: The


#GPT2-LoRA

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType, PeftModelForCausalLM, PeftModel


# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configure LoRA adapters
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"]
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Verify trainable LoRA parameters

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


#GPT2-DoRA

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from peft import get_peft_model, TaskType, PeftModelForCausalLM, PeftConfig
from peft import LoraConfig, AdaLoraConfig

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configure DoRA adapters
dora_config = AdaLoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,  # Low-rank matrix rank
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate
    target_modules=["c_attn", "c_proj"],  # Apply DoRA to attention layers
    beta1=0.85,  # Adaptive importance weighting factor
    beta2=0.85,  # Further control factor for importance weighting
    orth_reg_weight=0.5  # Orthogonality regularization weight
)

# Wrap the model with DoRA
model = get_peft_model(model, dora_config)
model.print_trainable_parameters()  # Verify trainable DoRA parameters

# Move model to device
model.to(device);


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/adalora/model.py:204: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 1,216,944 || all params: 125,656,788 || trainable%: 0.9685


#GPT2 AdaLoRA

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from peft import get_peft_model, TaskType, PeftModelForCausalLM, PeftConfig
from peft import AdaLoraConfig

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configure AdaLoRA adapters
adalora_config = AdaLoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,  # Low-rank matrix rank
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate
    target_modules=["c_attn", "c_proj"],  # Apply AdaLoRA to attention layers
    beta1=0.85,  # Adaptive importance weighting factor
    beta2=0.85,  # Further control factor for importance weighting
    orth_reg_weight=0.5  # Orthogonality regularization weight
)

# Wrap the model with AdaLoRA
model = get_peft_model(model, adalora_config)
model.print_trainable_parameters()  # Verify trainable AdaLoRA parameters

# Move model to device
model.to(device);


trainable params: 1,216,944 || all params: 125,656,788 || trainable%: 0.9685


#Multi-LoRA GTP2

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from peft import get_peft_model, LoraConfig, TaskType

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define multiple LoRA adapters for different model components
lora_configs = {
    "attention": LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=["c_attn", "c_proj"]
    ),
    "mlp": LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=4,
        lora_alpha=8,
        lora_dropout=0.05,
        target_modules=["mlp.c_fc", "mlp.c_proj"]
    )
}

# Apply modular LoRA to different layers
for adapter_name, config in lora_configs.items():
    model = get_peft_model(model, config)

# Verify trainable parameters
model.print_trainable_parameters()

# Move model to device
model.to(device);


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


#GPT2 VeLoRA:

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from peft import get_peft_model, TaskType, PeftModelForCausalLM, PeftConfig
from peft import LoraConfig

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configure VeLoRA adapters (Approximating VeLoRA with Variable Low-Rank Updates)
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,  # Initial low-rank matrix rank
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate
    target_modules=["c_attn", "c_proj"],  # Apply LoRA to attention layers
    rank_pattern={"c_attn": 16, "c_proj": 8},  # Simulating variable rank adjustment
)

# Wrap the model with LoRA (VeLoRA approximation)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Verify trainable VeLoRA parameters

# Move model to device
model.to(device);


trainable params: 1,105,920 || all params: 125,545,728 || trainable%: 0.8809


GPT2 MoRA

In [ ]:
!git clone https://github.com/kongds/MoRA.git

Cloning into 'MoRA'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 176 (delta 57), reused 136 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (176/176), 257.45 KiB | 4.95 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [ ]:
%cd MoRA
!pip -q install -e ./peft-mora

/content/MoRA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for peft (pyproject.toml) ... done


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from peft import get_peft_model, TaskType, PeftModelForCausalLM, PeftConfig, LoraConfig

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configure MoRA adapters
mora_config = LoraConfig(
    # enable MoRA
    use_mora=True,
    # type 1 (Sharing) for large lora ranks, Eq. 6 in paper
    # type 6 (RoPE based) for small lora ranks, Eq. 9 in paper
    mora_type=6,
    # lora rank here, we will calculate corresponding $\hat{r}$ in MoRA
    r=8,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"],
    task_type=TaskType.CAUSAL_LM,
)

# Wrap the model with MoRA
model = get_peft_model(model, mora_config)
model.print_trainable_parameters()  # Verify trainable MoRA parameters

# Move model to device
model.to(device);


trainable params: 800,544 || all params: 125,240,352 || trainable%: 0.6392061242370191
